In [1]:
from geopy.geocoders import Nominatim
import json
import folium
import requests

## Load Clubs data

In [2]:
with open("../scraper/data/clubs.json", "r") as in_file:
    clubs = json.load(in_file)

In [3]:
len(clubs)

418

In [4]:
clubs[0]["href"]

'/chelsea-fc/kader/verein/631/saison_id/2017'

In [5]:
'''
clubDict = {}
for club in clubs:
    club_id = club["href"].split("/")[4]
    if club_id not in clubDict:
        clubDict[club_id] = club
        
with open("../scraper/data/dictClubs.json", "w") as out:
    json.dump(clubDict, out)
'''

'\nclubDict = {}\nfor club in clubs:\n    club_id = club["href"].split("/")[4]\n    if club_id not in clubDict:\n        clubDict[club_id] = club\n        \nwith open("../scraper/data/dictClubs.json", "w") as out:\n    json.dump(clubDict, out)\n'

In [6]:
with open("../scraper/data/dictClubs.json", "r") as in_file:
    clubDict = json.load(in_file)

## Retrieve missing coordinates using GoogleMap requests

In [79]:
GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'

# For each club, we use its name and the stadium name to make a GoogleMap query and retrieve the stadium's coordinates
for club_id in clubDict.keys():
    club = clubDict[club_id]
    if "position" not in club:
        
        params = {
            'address': club["stadium"] + " " + club["name"]
        }
        # Do the request and get the response data
        req = requests.get(GOOGLE_MAPS_API_URL, params=params)
        res = req.json()
        # Use the first result
        if len(res['results']) > 0:
            result = res['results'][0]
            club["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

In [ ]:
# For each club, we use its name and the stadium name to make a GoogleMap query and retrieve the stadium's coordinates
for club_id in clubDict.keys():
    club = clubDict[club_id]
    if "position" not in club:
        
        params = {
            'address': club["stadium"]
        }
        # Do the request and get the response data
        req = requests.get(GOOGLE_MAPS_API_URL, params=params)
        res = req.json()
        # Use the first result
        if len(res['results']) > 0:
            result = res['results'][0]
            club["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

### Manual clean up

Check the clubs for which the coordinates couldn't be retrieved

In [ ]:
'''
for club_id in clubDict.keys():
    club = clubDict[club_id]
    if "position" not in club:
        print(str(club_id) + ": " + club["name"])
'''

Change manually the stadiums that couldn't be found with GoogleMaps.
We then rerun the google map coordinate retrieval

In [149]:
clubs[93]["stadium"] = 'S.B.V. Excelsior'
clubs[170]["stadium"] = 'Başakşehir Fatih Terim Stadium'
clubs[208]["stadium"] = 'Lokeren Daknamstadion'
clubs[209]["stadium"] = 'KAS Eupen'
clubs[221]["stadium"] = 'Obolon Arena'
clubs[235]["stadium"] = 'FC Vysočina Jihlava'
clubs[20]["stadium"] = "Juventus Stadium"

In [150]:
params = {
    'address': clubs[201]["stadium"] + " " + clubs[201]["name"]
}
# Do the request and get the response data
req = requests.get(GOOGLE_MAPS_API_URL, params=params)
res = req.json()
# Use the first result
if len(res['results']) > 0:
    result = res['results'][0]
    clubs[201]["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

## Save/Load Clubs with location data

In [160]:
'''
with open("../scraper/data/clubs_with_location.json", "w") as out:
    json.dump(clubs, out)
'''

'\nwith open("../scraper/data/clubs_with_location.json", "w") as out:\n    json.dump(clubs, out)\n'

In [8]:
with open("../scraper/data/clubs_with_location.json", "r") as in_file:
    clubs = json.load(in_file)

# Display clubs on map

In [46]:
m = folium.Map(location=[52, 15], zoom_start=4)
from folium.plugins import MarkerCluster

In [51]:
marker_cluster = MarkerCluster([]).add_to(m)

In [52]:
for club in clubs:
    folium.Marker(
        location=club["position"],
        popup=club["name"],
        icon=None,
    ).add_to(marker_cluster)

In [13]:
m

In [54]:
m.save('index.html')


In [15]:
html="""
    <h1> This is a big popup</h1><br>
    With a few lines of code...
    <p>
    <code>
        from numpy import *<br>
        exp(-2*pi)
    </code>
    </p>
    """
iframe = folium.element.IFrame(html=html, width=500, height=300)
popup = folium.Popup(iframe, max_width=2650)

folium.Marker([30,-100], popup=popup, clustered_marker = True).add_to(m)

TypeError: __init__() got an unexpected keyword argument 'clustered_marker'

In [16]:
m

In [35]:
icon = folium.features.CustomIcon("https://seeklogo.com/images/F/FC_Barcelona-logo-D941E13B46-seeklogo.com.png",
                                          icon_size=(24, 24))
marker = folium.Marker([0, 0], icon=icon, popup="FC Barcelona").add_to(m)

In [18]:
m.add_children(marker)

In [19]:
url = 'http://leafletjs.com/examples/custom-icons/{}'.format
icon_image = url('leaf-red.png')
shadow_image = url('leaf-shadow.png')

icon = folium.features.CustomIcon(
    icon_image,
    icon_size=(38, 95),
    icon_anchor=(22, 94),
    shadow_image=shadow_image,
    shadow_size=(50, 64),
    shadow_anchor=(4, 62),
    popup_anchor=(-3, -76)
)

marker = folium.Marker(
    location=[45.3288, -121.6625],
    icon=icon,
    popup='Mt. Hood Meadows'
)


m.add_child(marker)